In [1]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 110.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [5]:
import torch
from diffusers import StableDiffusionInstructPix2PixPipeline
from diffusers.utils import load_image
from PIL import Image as im
import requests
import io
import cv2
import numpy as np
import gradio as gr

API_URL = "https://api-inference.huggingface.co/models/ZB-Tech/Text-to-Image"
headers = {"Authorization": "Bearer hf_token"} #use you Hugging Face Access Key here!

model_id = "instruction-tuning-sd/cartoonizer"
pipeline = StableDiffusionInstructPix2PixPipeline.from_pretrained(
    model_id, torch_dtype=torch.float16, use_auth_token=True
  ).to("cuda")

def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.content

def cartoonizer(input_img,bg_prompt):
  if input_img is not None:
    data = im.fromarray(input_img)
    data = data.resize((300,300))
    org_image = load_image(data)
    cart_image = pipeline("Cartoonize the following image", image=org_image).images[0]
    if len(bg_prompt) !=0:
      image_bytes = query({
        "inputs": bg_prompt,
      })
    else:
      image_bytes = query({
        "inputs": "heaven",
      })
    bg_image = im.open(io.BytesIO(image_bytes))
    return [cart_image,bg_image]
  else:
    gr.Warning("Please upload an Input Image!")
    return [input_img,input_img]

with gr.Blocks(theme = gr.themes.Citrus()) as cart:
  gr.HTML("""<h1 align="center">Cartoonize your Image with best backgrounds!</h1>""")
  with gr.Tab("Cartoonize"):
    with gr.Row():
      image_input = gr.Image()
      image_output = gr.Image()
      text_img_output = gr.Image()

    txt_label = gr.Label("Enter your photo frame description:")
    txt_input = gr.Textbox()
    image_btn = gr.Button("Convert")

  image_btn.click(cartoonizer,inputs = [image_input,txt_input],outputs=[image_output,text_img_output])


cart.launch()


Keyword arguments {'use_auth_token': True} are not expected by StableDiffusionInstructPix2PixPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--instruction-tuning-sd--cartoonizer/snapshots/94ae1467bb3a3b231cb444ab0acd4295836014f1/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--instruction-tuning-sd--cartoonizer/snapshots/94ae1467bb3a3b231cb444ab0acd4295836014f1/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--instruction-tuning-sd--cartoonizer/snapshots/94ae1467bb3a3b231cb444ab0acd4295836014f1/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--instruction-tuning-sd--cartoonizer/snapshots/94ae1467bb3a3b231cb444ab0acd4295836014f1/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c941f4ee4b6dc47987.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
